In [4]:
!pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
  Using cached Shapely-1.7.1-1-cp39-cp39-manylinux1_x86_64.whl (1.0 MB)
  Using cached pyproj-3.2.1-cp39-cp39-manylinux2010_x86_64.whl (6.3 MB)
  Using cached Fiona-1.8.20-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (15.4 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)


In [5]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd

initial thoughts:

* For long trips (LA->San Jose, etc) 1 bus in the fleet per daily trip could be a reasonable assumption
* Medium trips (Stockton --> Sac, LA-->SD); .5 buses in fleet/trip?
* Short trips (SF Transbay), .3 buses in fleet/trip?

One validator per bus seems adequate since these are single-door, over-the-road coaches.

### Find stops for routes of interest (this seems to work fine)

In [6]:
amtrak_routes = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> collect())

In [7]:
ca_corridors = ['Capitol Corridor', 'Pacific Surfliner', 'San Joaquins']

In [8]:
ca_rail_routes = amtrak_routes >> filter(_.route_long_name.isin(ca_corridors))

In [9]:
ca_rail_routes = ca_rail_routes >> select(_.route_id, _.route_long_name)

In [10]:
ca_rail_trips = (tbl.gtfs_schedule.trips()
                >> filter(_.calitp_itp_id == 13)
                >> collect()
                >> inner_join(_, ca_rail_routes, on='route_id')
                )

In [11]:
ca_corridor_stops = (tbl.gtfs_schedule.stop_times()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.trip_id.isin(ca_rail_trips.trip_id))
                >> collect()
                >> distinct(_.stop_id, _keep_all = True)
                )

In [12]:
ca_corridor_stops.head(3)

,calitp_itp_id,calitp_url_number,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
0,13,0,7012818749,BFD,1,9:12:00,9:12:00,None,0,0,None,None,None,None,2021-10-19
1,13,0,5212816183,SAC,1,7:10:00,7:10:00,None,0,0,None,None,None,None,2021-10-19
2,13,0,7202808666,OKJ,1,10:12:00,10:12:00,None,0,0,None,None,None,None,2021-10-19


In [13]:
ca_thruway_trips = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.route_long_name == 'Amtrak Thruway Connecting Service')
                >> inner_join(_, tbl.gtfs_schedule.trips(), on='route_id')
                >> inner_join(_, tbl.gtfs_schedule.stop_times(), on='trip_id')
                >> filter(_.stop_id.isin(ca_corridor_stops.stop_id)) ## filter to trips connecting with CCJPA, LOSSAN and SJRRA
                >> collect()
                >> distinct(_.trip_id, _keep_all=True)
                >> select(_.calitp_itp_id, _.trip_id, _.route_id, _.service_id))

In [14]:
ca_thruway_trips

,calitp_itp_id,trip_id,route_id,service_id
0,13,49842815638,37329,2815638
1,13,54162815345,37329,2815345
2,13,66472814258,37329,2814258
3,13,66472814256,37329,2814256
4,13,54122815341,37329,2815341
...,...,...,...,...
858,13,66772814212,37329,2814212
859,13,66772814211,37329,2814211
860,13,33292814231,37329,2814231
861,13,50112817980,37329,2817980


In [15]:
amtrak_thursday_service = (
    tbl.views.gtfs_schedule_fact_daily_service()
    >> filter(_.calitp_itp_id == 13)
    >> filter(_.service_date == '2021-10-28') ## a Thursday
    >> collect())

In [16]:
thursday_thruway_trips = ca_thruway_trips >> filter(_.service_id.isin(amtrak_thursday_service.service_id))

In [17]:
thursday_thruway_trips.shape

(400, 4)

In [18]:
thursday_thruway_trips.head(10)

,calitp_itp_id,trip_id,route_id,service_id
1,13,54162815345,37329,2815345
2,13,66472814258,37329,2814258
6,13,66112814305,37329,2814305
14,13,54122815339,37329,2815339
15,13,36242819614,42917,2819614
20,13,40152820008,42895,2820008
23,13,49672819218,37329,2819218
25,13,63112816851,37329,2816851
30,13,54102815349,37329,2815349
38,13,63132816849,37329,2816849


In [19]:
thursday_thruway_trips['route_id'].value_counts()

37329    380
11317      8
42896      6
42895      4
42917      2
Name: route_id, dtype: int64

#### 380 daily trips for route_id 37329 seems like an error. Also that ID seems to serve many routes?

In [20]:
thursday_thruway_stop_times = (tbl.gtfs_schedule.stop_times()
                               >> filter(_.calitp_itp_id == 13)
                               >> filter(_.trip_id.isin(thursday_thruway_trips.trip_id))
                               >> collect()
                               >> inner_join(_, thursday_thruway_trips, on='trip_id'))

## Seems to be way off (San Jose and Stockton are not served hundreds of times/day by Thruway buses...)

In [21]:
thursday_thruway_stop_times >> count(_.route_id, _.stop_id, sort=True)

,route_id,stop_id,n
0,37329,SJC,274
1,37329,SKN,199
2,37329,FRT,182
3,37329,TRA,182
4,37329,DBP,182
...,...,...,...
134,37329,SIM,1
135,37329,CPN,1
136,37329,CML,1
137,37329,SNC,1


### Sandbox

In [22]:
(tbl.gtfs_schedule.trips()
                >> filter(_.calitp_itp_id == 13)
                >> count(_.route_id))

,route_id,n
0,37329,917
1,88,360
2,71,218
3,40751,199
4,84,169


In [23]:
route_37329 = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.route_long_name == 'Amtrak Thruway Connecting Service')
                >> collect())

In [24]:
(tbl.gtfs_schedule.shapes()
                >> filter(_.calitp_itp_id == 13))

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at
0,13,0,3,44.521053,-73.394035,315,None,2021-10-19
1,13,0,3,43.811103,-73.394451,827,None,2021-10-19
2,13,0,3,42.258361,-73.795262,1644,None,2021-10-19
3,13,0,4,42.787777,-73.939269,1244,None,2021-10-19
4,13,0,4,43.208691,-73.635209,1501,None,2021-10-19


In [25]:
ca_thruway_trips >> filter(_.service_id.isin(['2814305']))

,calitp_itp_id,trip_id,route_id,service_id
6,13,66112814305,37329,2814305


In [31]:
df = (tbl.views.gtfs_schedule_fact_daily_service()
    >> filter(_.calitp_itp_id == 13)
    >> filter(_.service_date == '2021-10-28') ## a Thursday
    >> inner_join(_, tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == 13), on = 'service_id')
    >> filter(_.route_id == '37329')
    >> collect())
df

,feed_key,calitp_itp_id_x,calitp_url_number_x,service_date,service_id,ttl_service_hours,n_trips,n_routes,first_departure_ts,last_arrival_ts,...,route_id,direction_id,bikes_allowed,block_id,shape_id,trip_id,calitp_itp_id_y,calitp_extracted_at,calitp_url_number_y,trip_short_name
0,-1246851374109274803,13,0,2021-10-28,2813905,2.000000,1,1,54900,62100,...,37329,0,None,None,None,56222813905,13,2021-10-19,0,5622
1,-1246851374109274803,13,0,2021-10-28,2814189,2.000000,1,1,43500,50700,...,37329,1,None,None,None,38122814189,13,2021-10-19,0,3812
2,-1246851374109274803,13,0,2021-10-28,2819224,2.000000,1,1,58200,65400,...,37329,1,None,None,None,38162819224,13,2021-10-19,0,3816
3,-1246851374109274803,13,0,2021-10-28,2814174,2.250000,1,1,33000,41100,...,37329,0,None,None,None,39102814174,13,2021-10-19,0,3910
4,-1246851374109274803,13,0,2021-10-28,2818589,2.250000,1,1,30000,38100,...,37329,1,None,None,None,65102818589,13,2021-10-19,0,6510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,-1246851374109274803,13,0,2021-10-28,2817681,1.416667,1,1,74700,79800,...,37329,0,None,None,None,65152817681,13,2021-10-19,0,6515
413,-1246851374109274803,13,0,2021-10-28,2819231,1.666667,1,1,67800,73800,...,37329,1,None,None,None,65182819231,13,2021-10-19,0,6518
414,-1246851374109274803,13,0,2021-10-28,2819214,1.916667,1,1,85500,92400,...,37329,0,None,None,None,38192819214,13,2021-10-19,0,3819
415,-1246851374109274803,13,0,2021-10-28,2813788,3.716667,1,1,20400,33780,...,37329,1,None,None,None,63322813788,13,2021-10-19,0,6332


In [27]:
amtrak_thursday_service

,feed_key,calitp_itp_id,calitp_url_number,service_date,service_id,ttl_service_hours,n_trips,n_routes,first_departure_ts,last_arrival_ts,service_window
0,-1246851374109274803,13,0,2021-10-28,2813905,2.000000,1,1,54900,62100,2.000000
1,-1246851374109274803,13,0,2021-10-28,2814189,2.000000,1,1,43500,50700,2.000000
2,-1246851374109274803,13,0,2021-10-28,2815144,2.000000,1,1,48000,55200,2.000000
3,-1246851374109274803,13,0,2021-10-28,2819224,2.000000,1,1,58200,65400,2.000000
4,-1246851374109274803,13,0,2021-10-28,2819454,2.000000,1,1,86400,93600,2.000000
...,...,...,...,...,...,...,...,...,...,...,...
1012,-1246851374109274803,13,0,2021-10-28,2809946,2.866667,1,1,64680,75000,2.866667
1013,-1246851374109274803,13,0,2021-10-28,2808673,3.116667,1,1,72300,83520,3.116667
1014,-1246851374109274803,13,0,2021-10-28,2820157,5.983333,1,1,32100,53640,5.983333
1015,-1246851374109274803,13,0,2021-10-28,2809400,7.983333,1,1,55200,83940,7.983333


In [29]:
tbl.views.gtfs_schedule_fact_daily_trips() >> filter(_.calitp_itp_id == 13)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 10208935936 or higher required.

(job ID: 877b8ecc-e64b-419b-b46e-873c02d46159)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 10208935936 or higher required.

(job ID: 1787c777-8e92-4759-9cb4-6aa3d9dccd94)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          